Artificial Intelligence - Assigment No.2 - Supervised Learning


About the Dataset

The dataset that we are going to use in this project is the "BCCC-cPacket-Cloud-DDoS-2024", a cybersecurity dataset to train, test and analyze cybersecurity models. It includes equal number of malicious and benign DoH(Domain Over HTTP) network traffic instances, with 249,836 instances in each category. It has 28 attributes, being them all numeric-valued measurements, regarding about the packets length, response time, byte flow, etc. And the last one, Label, which labels the packet as Benign or Malicious.

About the Problem

Domain Name System is one of the early and vulnerable network protocols which has several security loopholes, because of that has always been an area of great concern for cybersecurity researches. To overcome some of the DoH problems and vulnerabilities, our model will help us undestand in a better/depper way what makes a packet benign and malicious. So this is a binary classification problem where the target variable is the label of benign or malicious being applied to a DoH packet.

About the Solution

The solution for this problem was a supervised learning model, we splitted the above mentioned dataset within a tranning part
(70%) and a testing part(30%). The model will be evaluated using the x1, x2, x3 metrics.


Importing Libraries

To start we need to install all the libraires that are going to be used in this project, they are all located in the "requirements.txt" file and can easily installed with the below command 

	pip install -r requirements.txt

In [13]:
import os
import sys
#!{sys.executable} -m pip install numpy
import numpy as np
#!{sys.executable} -m pip install pandas
import pandas as pd
#!{sys.executable} -m pip install scikit-learn
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import time

Loading and describing the dataset with pandas module

In [14]:
df = pd.read_csv('archive\\BCCC-CIRA-CIC-DoHBrw-2020.csv')
df.head()
df.describe()

,FlowBytesSent,FlowSentRate,FlowBytesReceived,FlowReceivedRate,PacketLengthVariance,PacketLengthStandardDeviation,PacketLengthMean,PacketLengthMedian,PacketLengthMode,PacketLengthSkewFromMedian,...,PacketTimeSkewFromMode,PacketTimeCoefficientofVariation,ResponseTimeTimeVariance,ResponseTimeTimeStandardDeviation,ResponseTimeTimeMean,ResponseTimeTimeMedian,ResponseTimeTimeMode,ResponseTimeTimeSkewFromMedian,ResponseTimeTimeSkewFromMode,ResponseTimeTimeCoefficientofVariation
count,4.991060e+05,4.991060e+05,4.991060e+05,4.991060e+05,4.991060e+05,499106.000000,499106.000000,499106.000000,499106.000000,499106.000000,...,499106.000000,499106.000000,499106.000000,499106.000000,499106.000000,499106.000000,499106.000000,499106.000000,499106.000000,499106.000000
mean,4.020093e+04,4.733915e+04,4.250156e+04,3.166829e+04,9.263585e+04,220.169083,173.159814,95.472702,70.749953,0.442934,...,1.246052,0.972995,1.711825,0.319586,0.442021,0.394671,0.205894,-0.968984,-0.060800,1.114611
std,1.439617e+05,4.212751e+05,1.393927e+05,2.566805e+05,1.534938e+05,210.141825,85.506490,32.999740,14.830277,1.551998,...,0.713803,0.520212,11.106412,1.244031,2.071807,2.351233,1.724873,3.163775,3.198770,1.735820
min,5.500000e+01,1.464903e+00,5.400000e+01,1.576245e+00,0.000000e+00,0.000000,56.000000,54.000000,54.000000,-10.000000,...,-5.265523,0.077182,0.000000,0.000000,0.000005,0.000002,-0.000001,-10.000000,-10.000000,0.000000
25%,6.180000e+02,5.410781e+01,4.760000e+02,1.418128e+02,4.692099e+02,21.661253,92.000000,76.000000,66.000000,0.199848,...,0.627988,0.577836,0.000021,0.004454,0.010538,0.012238,0.000016,-1.797168,0.393686,0.552687
50%,1.807000e+03,3.640969e+02,4.827000e+03,4.611146e+02,1.826789e+04,135.142971,152.488283,87.000000,68.000000,0.986397,...,1.224949,0.748184,0.000079,0.008820,0.015148,0.015407,0.000034,0.000000,0.908002,0.800649
75%,5.542000e+03,3.810260e+03,7.888000e+03,4.215498e+03,1.415989e+05,376.296309,228.758621,105.000000,68.000000,1.201609,...,1.716577,1.516263,0.000357,0.018640,0.024903,0.016300,0.015161,0.936908,1.305393,1.208105
max,8.015359e+06,2.304348e+07,7.723184e+06,7.600000e+06,1.578115e+06,1256.230616,689.800000,317.000000,553.000000,2.932375,...,12.956406,5.616085,647.245330,25.441017,28.017596,28.017596,28.017596,2.970716,5.428781,66.309747


Data Preprocessing

In [28]:
# Checking for NA values, that is, missing/empty values.
df.isna().sum() 

FlowBytesSent                             0
FlowSentRate                              0
FlowBytesReceived                         0
FlowReceivedRate                          0
PacketLengthVariance                      0
PacketLengthStandardDeviation             0
PacketLengthMean                          0
PacketLengthMedian                        0
PacketLengthMode                          0
PacketLengthSkewFromMedian                0
PacketLengthSkewFromMode                  0
PacketLengthCoefficientofVariation        0
PacketTimeVariance                        0
PacketTimeStandardDeviation               0
PacketTimeMean                            0
PacketTimeMedian                          0
PacketTimeMode                            0
PacketTimeSkewFromMedian                  0
PacketTimeSkewFromMode                    0
PacketTimeCoefficientofVariation          0
ResponseTimeTimeVariance                  0
ResponseTimeTimeStandardDeviation         0
ResponseTimeTimeMean            

As we can see, no missing values were found